In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [6]:
# ID da planilha
sheet_id = '1s9AzlV2unNyPs5EM3kBXp61iMpe6p3cO0xNeTXEOVLI'
# IDs das abas
tab_ids = {'Offers':'0',
           'Bids':'1696916279',
           'SF': '2136104990',
           'Network': '1354044027'}
offers = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Offers"]}')

bids = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Bids"]}')

SF = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["SF"]}', header=None)
SF = SF.replace(',', '.', regex=True).astype(float)
SF = np.array(SF)

network = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={tab_ids["Network"]}')

In [7]:
num_offers = len(offers)
num_bids = len(bids)
num_lines = len(network)
unique_locations = set(offers['Barra']).union(set(bids['Barra']))
num_nodes = len(unique_locations)

#Modelo Concreto
model = ConcreteModel()

#Set das Barras do Sistema
model.barras = Set(initialize=sorted(unique_locations))

#Set Linhas do Sistema
linhas = [f'Linha {i+1}' for i in range(num_lines)]
model.linhas = Set(initialize=linhas)

#Set Lances Vendedores
lances_vendedores = [f'Lance {i+1}' for i in range(num_offers)]
model.offers = Set(initialize=lances_vendedores)

#Set Lances Compradores
lances_compradores = [f'Lance {i+1}' for i in range(num_bids)]
model.bids = Set(initialize=lances_compradores)

#Parâmetros de Linha
barra_de = network['Barra de'].tolist()
model.barra_de = Param(model.linhas, initialize={linhas[i]: barra_de[i] for i in range(num_lines)}, within=model.barras)

barra_para = network['Barra para'].tolist()
model.barra_para = Param(model.linhas, initialize={linhas[i]: barra_para[i] for i in range(num_lines)}, within=model.barras)

fmax = np.array(network['F_max'].tolist())
model.fmax = Param(model.linhas, initialize={linhas[i]: network.loc[i, "F_max"] for i in range(num_lines)})
model.fmax.pprint()

#Parâmetros dos Lances Compradores
offers_bar = offers['Barra'].tolist()
model.offers_bar = Param(model.offers, initialize={lances_vendedores[i]: offers_bar[i] for i in range(num_offers)}, within=model.barras)
model.offers_bar.pprint()

offers_price = offers['Price'].tolist()
model.offers_price = Param(model.offers, initialize={lances_vendedores[i]: offers_price[i] for i in range(num_offers)})
model.offers_price.pprint()

#Parâmetros dos Lances Vendedores
bids_bar = bids['Barra'].tolist()
model.bids_bar = Param(model.bids, initialize={lances_compradores[i]: bids_bar[i] for i in range(num_bids)}, within=model.barras)
model.bids_bar.pprint()

bids_price = bids['Price'].tolist()
model.bids_price = Param(model.bids, initialize={lances_compradores[i]: bids_price[i] for i in range(num_bids)})
model.bids_price.pprint()

#Limite das variáveis
selling_max = offers['Qty'].tolist()
selling_bounds = {lances_vendedores[i]: (0, selling_max[i]) for i in range(num_offers)}

buying_max = bids['Qty'].tolist()
buying_bounds = {lances_compradores[i]: (0, buying_max[i]) for i in range(num_bids)}

#Variáveis de potência vendida de cada lance vendedor
model.sell_qty = Var(model.offers, bounds=selling_bounds, domain=Reals)
model.sell_qty.display()

#Variáveis de potência comprada de cada lance comprador
model.buy_qty = Var(model.bids, bounds=buying_bounds, domain=Reals)
model.buy_qty.display()

#Declarando Função Objetivo
def FOB(model):
    receita = sum(model.sell_qty[i]*model.offers_price[i] for i in model.offers)
    despesa = sum(model.buy_qty[j]*model.bids_price[j] for j in model.bids)
    return despesa-receita # Maximizar a diferença entre área compradora e vendedora

model.custo_total = Objective(rule=FOB, sense=maximize)

def potencia_injetada(model):
    liq_injection = []
    for barra in model.barras:
        pot_vendida = sum(model.sell_qty[i] for i in model.offers if model.offers_bar[i] == barra)
        pot_comprada = sum(model.buy_qty[j] for j in model.bids if model.bids_bar[j] == barra)
        pot_injetada = pot_vendida - pot_comprada
        liq_injection.append(pot_injetada)
    return liq_injection

def fluxo(model, SF):
    # Função para calcular o fluxo em cada linha como uma expressão Pyomo
    pot_inj = np.array(potencia_injetada(model))
    return np.dot(SF, pot_inj)

#Restrição de Fluxo:
model.restr_desigualdade = ConstraintList()
for i in range(num_lines):
    model.restr_desigualdade.add(fluxo(model, SF)[i] >= -fmax[i])
    model.restr_desigualdade.add(fluxo(model, SF)[i] <= fmax[i])
model.restr_desigualdade.pprint()

#Balanço de Potência do Sistema
model.restr_igualdade = ConstraintList()
model.restr_igualdade.add(expr=sum(model.buy_qty[i] for i in model.bids) - sum(model.sell_qty[i] for i in model.offers) == 0)

solver = SolverFactory('glpk')
# Adicionar o componente para armazenar duais
model.dual = Suffix(direction=Suffix.IMPORT)

results = solver.solve(model, tee=False)

fmax : Size=6, Index=linhas, Domain=Any, Default=None, Mutable=False
    Key     : Value
    Linha 1 :   100
    Linha 2 :   100
    Linha 3 :   100
    Linha 4 :   100
    Linha 5 :   100
    Linha 6 :   100
offers_bar : Size=11, Index=offers, Domain=barras, Default=None, Mutable=False
    Key      : Value
     Lance 1 :     A
    Lance 10 :     D
    Lance 11 :     D
     Lance 2 :     A
     Lance 3 :     B
     Lance 4 :     B
     Lance 5 :     C
     Lance 6 :     C
     Lance 7 :     C
     Lance 8 :     D
     Lance 9 :     D
offers_price : Size=11, Index=offers, Domain=Any, Default=None, Mutable=False
    Key      : Value
     Lance 1 :     5
    Lance 10 :    30
    Lance 11 :    18
     Lance 2 :    15
     Lance 3 :     6
     Lance 4 :     7
     Lance 5 :     6
     Lance 6 :    15
     Lance 7 :    25
     Lance 8 :    10
     Lance 9 :    15
bids_bar : Size=7, Index=bids, Domain=barras, Default=None, Mutable=False
    Key     : Value
    Lance 1 :     E
    Lance 2 :   

In [8]:
# Relatório dos resultados de otimização
print('Status Final do Problema de Otimização:', results.solver.status, '\n')
print('Condição de Término:', results.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.custo_total), '\n')

# Exibindo as variáveis de potência vendida de cada lance vendedor
print("Resultados das Potências Vendidas (Lances dos Vendedores):")
venda_total = 0
for lance in model.offers:
    venda_total += model.sell_qty[lance].value*model.offers_price[lance]
    print(f"{lance}: {model.sell_qty[lance].value} MWh")
print(f'Venda total:{venda_total}$')

# Exibindo as variáveis de potência comprada de cada lance comprador
compra_total = 0
print("\nResultados das Potências Compradas (Lances dos Compradores):")
for lance in model.bids:
    compra_total += model.buy_qty[lance].value*model.bids_price[lance]
    print(f"{lance}: {model.buy_qty[lance].value} MWh")
print(f'Compra total: R${compra_total}')

# Obter o custo marginal (multiplicador de Lagrange) da restrição de igualdade
custo_marginal = model.dual[model.restr_igualdade[1]]
print("Custo Marginal: R$", custo_marginal)

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 4275.0 

Resultados das Potências Vendidas (Lances dos Vendedores):
Lance 1: 10.0 MWh
Lance 2: 20.0 MWh
Lance 3: 15.0 MWh
Lance 4: 40.0 MWh
Lance 5: 30.0 MWh
Lance 6: 35.0 MWh
Lance 7: 0.0 MWh
Lance 8: 10.0 MWh
Lance 9: 20.0 MWh
Lance 10: 0.0 MWh
Lance 11: 0.0 MWh
Venda total:1825.0$

Resultados das Potências Compradas (Lances dos Compradores):
Lance 1: 20.0 MWh
Lance 2: 0.0 MWh
Lance 3: 25.0 MWh
Lance 4: 45.0 MWh
Lance 5: 50.0 MWh
Lance 6: 0.0 MWh
Lance 7: 40.0 MWh
Compra total: R$6100.0
Custo Marginal: R$ 15.0
